# Automating Machine Learning for Networking

This notebook is designed to give you a **very** simple example of how to use [nPrint](https://nprint.github.io/nprint/) in a generic machine learning pipeline and the rapid pace at which we can train new models and test new ideas on network traffic. Note that this example is simply to show the pipeline, not to test a hard problem. The traffic collected is to the same website over the course of about 15 seconds.

## Learning Objectives

In this brief overview, you will use the nprint tool to generate fingerprints from packet captures (pcaps) that can be input to a variety of machine learning algorithms. By the end of this activity you will:

1. Understand how to take a packet trace and represent it in a standard, generic form.
2. Run nprint on a few classes of machine learning algorithms.
3. Try the pipeline on some of your own (labeled) packet traces to understand how to build an end-to-end machine learning pipeline for a cybersecurity problem.

## Tasks

1. Install nprint and run it on a packet trace---either the one provided, one that is available from public repositories (e.g., CICIDS). Optionally, try it on your own traffic.
2. Run the notebook below to understand nprint and the basic machine learning pipeline.
3. Try nprint on a new (security) classification problem, using one of the provided datasets.

---

## Requirements

nPrint must be installed into $PATH for external commands to work. Note: You may not be able to do this part in Google collab; it may only work if you have a local (Linux) machine on which you are running the notebook.  If that is the case, the second cell where you execute the commands on pcaps may not run, but we have provided the "npt" nprint output files as well, so you can run the rest of the notebook.

You will want to install:

* nPrint ([Installation instructions](https://github.com/nprint/nprint))
* pcapML ([Installation instructions](https://github.com/nprint/pcapml))


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

## Part 1: Generating nPrints from Traffic

First, use nPrint to generate nPrints from each traffic trace, *only including the TCP headers in the nPrints*. (see which option you will need for that).

In [2]:
nprint = '/usr/local/bin/nprint'
data = 'data/'

cmd_80 = '{} -P {}/port80.pcap -t -W {}/port80.npt'.format(nprint, data, data)
cmd_443 = '{} -P {}/port443.pcap -t -W {}/port443.npt'.format(nprint, data, data)
!{cmd_80}
!{cmd_443}

tcsh(44085,0x203109e00) malloc: can't allocate region
:*** mach_vm_map(size=1048576, flags: 100) failed (error code=268435465)
tcsh(44085,0x203109e00) malloc: *** set a breakpoint in malloc_error_break to debug
Out of memory


Lets examine the nPrints, which can be directly loaded with Pandas. Load the nprints using `read_csv` function in Pandas into data frames. How many packets are in each nprint? How many features are in each packet?

In [12]:
import pandas as pd

nprint_80 = pd.read_csv('{}/port80.npt'.format(data), index_col=0)
nprint_443 = pd.read_csv('{}/port443.npt'.format(data), index_col=0)

print('Port 80 nPrint: Number of Packets: {0}, Features per packet: {1}'.format(nprint_80.shape[0], nprint_80.shape[1]))
print('Port 443 nPrint: Number of Packets: {0}, Features per packet: {1}'.format(nprint_443.shape[0], nprint_443.shape[1]))

Port 80 nPrint: Number of Packets: 2421, Features per packet: 480
Port 443 nPrint: Number of Packets: 2500, Features per packet: 480


You should see that each nprint has the same number of features, which is the maximum number of bits in a TCP header. Let's look at the header itself.

Notice how each bit (feature) is named according to the exact bit it represents in the packet, and all the possible bits of a TCP header are accounted for.

In [13]:
print(nprint_80.columns)
print(nprint_443.columns)

Index(['tcp_sprt_0', 'tcp_sprt_1', 'tcp_sprt_2', 'tcp_sprt_3', 'tcp_sprt_4',
       'tcp_sprt_5', 'tcp_sprt_6', 'tcp_sprt_7', 'tcp_sprt_8', 'tcp_sprt_9',
       ...
       'tcp_opt_310', 'tcp_opt_311', 'tcp_opt_312', 'tcp_opt_313',
       'tcp_opt_314', 'tcp_opt_315', 'tcp_opt_316', 'tcp_opt_317',
       'tcp_opt_318', 'tcp_opt_319'],
      dtype='object', length=480)
Index(['tcp_sprt_0', 'tcp_sprt_1', 'tcp_sprt_2', 'tcp_sprt_3', 'tcp_sprt_4',
       'tcp_sprt_5', 'tcp_sprt_6', 'tcp_sprt_7', 'tcp_sprt_8', 'tcp_sprt_9',
       ...
       'tcp_opt_310', 'tcp_opt_311', 'tcp_opt_312', 'tcp_opt_313',
       'tcp_opt_314', 'tcp_opt_315', 'tcp_opt_316', 'tcp_opt_317',
       'tcp_opt_318', 'tcp_opt_319'],
      dtype='object', length=480)


## Part 2: nPrint to Machine Learning Samples

Now we need to take each nPrint and make each packet a "sample" for the machine learning task at hand. In this case, we'll set up a supervised learning task where port 80 traffic is labeled "unencrypted" and port 443 traffic is labeled "encrypted"

In [14]:
import numpy as np

def label_data(data, label, features, labels):
    for _, row in data.iterrows():
        features.append(np.array(row))
        labels.append(label)
    return features, labels

def train_eval(features,labels,clf):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(features, labels)

    clf.fit(X_train, y_train) 
    y_pred = clf.predict(X_test)

    # Statistics
    report = classification_report(y_test, y_pred)
    print(report)

    # Let's also get the ROC AUC score while we're here, which requires a probability instead of just the prediction
    y_pred_proba = clf.predict_proba(X_test)
    # predict_proba gives us a probability estimate of each class, while roc_auc just cares about the "positive" class
    y_pred_proba_pos = [sublist[1] for sublist in y_pred_proba]
    roc = roc_auc_score(y_test, y_pred_proba_pos)
    print('ROC AUC Score: {0}'.format(roc))

def eval_nprint(class1, class2):
    
    (cmd1, label1) = class1
    (cmd2, label2) = class2
    
    # Generate nPrints
    !{cmd1}
    !{cmd2}

    # Load nPrints
    df1 = pd.read_csv('{}/port80.npt'.format(data), index_col=0)
    df2 = pd.read_csv('{}/port443.npt'.format(data), index_col=0)

    features = []
    labels = []

    (features, labels) = label_data(df1, label1, features, labels)
    (features, labels) = label_data(df2, label2, features, labels)

    rf = RandomForestClassifier(n_estimators=1000, max_depth=None, min_samples_split=2, random_state=0)
    train_eval(features,labels,rf)
    return rf

## Part 3: Training a Classifier

We're already ready to train and test a model on the traffic we gathered. Let's split the data into training and testing data, train a model, and get a stat report.

In [16]:
# Initialize Classifier, Train, and Predict
rf = eval_nprint((cmd_80,'unencrypted'), (cmd_443, 'encrypted'))

              precision    recall  f1-score   support

   encrypted       1.00      1.00      1.00       634
 unencrypted       1.00      1.00      1.00       597

    accuracy                           1.00      1231
   macro avg       1.00      1.00      1.00      1231
weighted avg       1.00      1.00      1.00      1231

ROC AUC Score: 1.0


### Understanding the Model

nPrint's alignment of each packet allows for understanding the specific features (parts of the packet) that are driving the model's performance. It turns out that the options that are being set in the TCP header is actually more important than the port numbers themselves!

## Part 4: Exploring Different Representations

Now that we have a generic pipeline, we can leverage nPrint's flags to generate different versions of nPrints. 

Test a version of this classification problem using **only** the IPv4 Headers of the packets.

In [19]:
rf = eval_nprint((cmd_80,'unencrypted'), (cmd_443, 'encrypted'))

              precision    recall  f1-score   support

   encrypted       1.00      1.00      1.00       623
 unencrypted       1.00      1.00      1.00       608

    accuracy                           1.00      1231
   macro avg       1.00      1.00      1.00      1231
weighted avg       1.00      1.00      1.00      1231

ROC AUC Score: 1.0


How about Testing using just the first 30 payload bytes in each packet?

In [20]:
cmd_80 = '{} -P {}/port80.pcap -p 30 -W {}/port80.npt'.format(nprint, data, data)
cmd_443 = '{} -P {}/port443.pcap -p 30 -W {}/port443.npt'.format(nprint, data, data)

rf = eval_nprint((cmd_80,'unencrypted'), (cmd_443, 'encrypted'))

              precision    recall  f1-score   support

   encrypted       0.67      0.68      0.67       635
 unencrypted       0.65      0.64      0.65       596

    accuracy                           0.66      1231
   macro avg       0.66      0.66      0.66      1231
weighted avg       0.66      0.66      0.66      1231

ROC AUC Score: 0.6959493737779421


Using this representation performs much more poorly. Why might that be the case?

## Summary

Hopefully this gives you a better idea of how nPrint can be used to rapidly train and test models for different traffic analysis problems. While this problem was contrived and simple, the same basic steps can be performed for any single-packet classification problem. 

If you want to train and test using **sets** of packets as input to a model, you'll either need a model that can handle that input, such as a CNN, or to flatten the 2D packet sample into a 1d sample for use with a model such as the random forest above.